## Mini Project III

In [124]:
import pandas as pd
import numpy as np
import seaborn as sns

# Cleaning

Create two separate customer segmentations (using clustering) to split them into 3-5 clusters

In [125]:
# Get data for customers

customers = pd.read_csv('data/twm_customer.csv', delimiter=';')


In [126]:
customers.head()

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
0,1362691,26150,46,5,1,M,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV
1,1362487,6605,71,1,0,M,2,,ChingDyi ...,Moussavi ...,10603,Daffodil ...,90159,Los Angeles,CA
2,1363160,18548,38,8,0,F,1,,Rosa ...,Johnston ...,8817,Figueroa ...,90024,Los Angeles,CA
3,1362752,47668,54,3,0,F,1,,Lisa ...,Martin ...,676,Humble ...,90172,Los Angeles,CA
4,1362548,44554,59,9,2,F,4,,Barbara ...,O'Malley ...,6578,C ...,10138,New York City,NY


In [127]:
# remove useless columns or lots of null values
customers = customers.drop(['name_prefix', 'first_name', 'last_name', 'street_name', 'street_nbr', 'postal_code', 'state_code'], axis=1)
customers.describe()

,cust_id,income,age,years_with_bank,nbr_children,marital_status
count,7.470000e+02,747.000000,747.000000,747.000000,747.000000,747.000000
mean,1.362991e+06,22728.281124,42.479250,3.907631,0.714859,1.882195
std,2.925255e+02,22207.221405,19.114879,2.675634,1.103410,0.892051
min,1.362480e+06,0.000000,13.000000,0.000000,0.000000,1.000000
25%,1.362732e+06,7118.500000,28.000000,2.000000,0.000000,1.000000
50%,1.362993e+06,17242.000000,42.000000,4.000000,0.000000,2.000000
75%,1.363244e+06,31338.000000,56.000000,6.000000,1.000000,2.000000
max,1.363495e+06,144157.000000,89.000000,9.000000,5.000000,4.000000


In [128]:
# Check for duplicate ids and remove them
idsUnique = len(set(customers.cust_id))
idsTotal = customers.shape[0]
idsdupe = idsTotal - idsUnique
print(idsdupe)



0


In [129]:
# missing data
def checkNull(data):
    """Check if data in frame is null"""
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['total', 'percent'])
    return missing_data.head(20)

missing_data = checkNull(customers)
missing_data

,total,percent
cust_id,0,0.0
income,0,0.0
age,0,0.0
years_with_bank,0,0.0
nbr_children,0,0.0
gender,0,0.0
marital_status,0,0.0
city_name,0,0.0


# Encoding

In [130]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cust_id          747 non-null    int64 
 1   income           747 non-null    int64 
 2   age              747 non-null    int64 
 3   years_with_bank  747 non-null    int64 
 4   nbr_children     747 non-null    int64 
 5   gender           747 non-null    object
 6   marital_status   747 non-null    int64 
 7   city_name        747 non-null    object
dtypes: int64(6), object(2)
memory usage: 46.8+ KB


In [131]:
customers.head()


,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,city_name
0,1362691,26150,46,5,1,M,2,Las Vegas
1,1362487,6605,71,1,0,M,2,Los Angeles
2,1363160,18548,38,8,0,F,1,Los Angeles
3,1362752,47668,54,3,0,F,1,Los Angeles
4,1362548,44554,59,9,2,F,4,New York City


In [132]:
# encode simple categorical

customers = customers.replace({
                                'gender': {'F ': 0, 'M ': 1}
                                })
customers

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,city_name
0,1362691,26150,46,5,1,1,2,Las Vegas
1,1362487,6605,71,1,0,1,2,Los Angeles
2,1363160,18548,38,8,0,0,1,Los Angeles
3,1362752,47668,54,3,0,0,1,Los Angeles
4,1362548,44554,59,9,2,0,4,New York City
...,...,...,...,...,...,...,...,...
742,1363324,14795,36,6,1,0,4,Los Angeles
743,1362895,26387,56,6,1,1,2,New York City
744,1362569,61300,50,0,2,1,2,Albuquerque
745,1363364,15100,37,7,0,0,2,Birmingham


In [133]:
# get dummy values for cities

dummy_cities = pd.get_dummies(customers.city_name)
customers = pd.concat([customers,dummy_cities], axis=1)

In [134]:
# drop cat cities

customers = customers.drop('city_name', axis=1)

In [135]:
customers

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,Albuquerque,Anaheim,Arlington,...,Seattle,St Louis,St Paul,Tampa,Toledo,Tucson,Tulsa,Virginia Beach,Washington,Wichita
0,1362691,26150,46,5,1,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1362487,6605,71,1,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1363160,18548,38,8,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1362752,47668,54,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1362548,44554,59,9,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,1363324,14795,36,6,1,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
743,1362895,26387,56,6,1,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
744,1362569,61300,50,0,2,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
745,1363364,15100,37,7,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
customers.to_csv(path_or_buf='customers.csv', sep=';')